In [1]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 2.1 MB/s eta 0:00:0000:0100:010m
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install nest_asyncio

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install asyncpg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 1.6 MB/s eta 0:00:0000:0100:010m
Note: you may need to restart the kernel to use updated packages.


In [54]:
import getpass
import aiohttp
import asyncpg
import asyncio
import psycopg2
from psycopg2 import pool
from os import getenv
from dotenv import load_dotenv
from urllib.parse import quote
from difflib import unified_diff, SequenceMatcher
import nest_asyncio
import json
import pandas as pd
from IPython.display import display, HTML
from sqlalchemy import create_engine

load_dotenv(override=True)
DM_USER = getenv('DM_USER')
DM_PASS = quote(getenv('DM_PASS'))
SYS_NAMES = ["prelude", "prelude-masks"] 
DS_IDS = {'prelude': {'exp_ids': 1425, 'summary': 1426}, 'prelude-masks' : {'exp_ids': 1422, 'summary': 1423}}
BASE_URL = 'dotmatics.net/browser/api'
LIMIT = 15
PROJECT_ID = 100000
DB_POOL = None
# library allows you to run nested event loops, works for Jupyter
nest_asyncio.apply()
DB_CONFIG = {
    "DB_NAME": getenv("DB_NAME"),
    "DB_USER": getenv("DB_USER"),
    "DB_PASS": getenv("DB_PASS"),
    "DB_HOST": getenv("DB_HOST"),
    "DB_PORT": getenv("DB_PORT"),
}

In [2]:
async def init_db():
    """
    Initializes the database connection pool.
    """
    global DB_POOL
    DB_POOL = await asyncpg.create_pool(
        user=DB_CONFIG['DB_USER'],
        password=DB_CONFIG['DB_PASS'],
        database=DB_CONFIG['DB_NAME'],
        host=DB_CONFIG['DB_HOST'],
        port=DB_CONFIG['DB_PORT'],
        min_size=1,
        max_size=10
    )

In [35]:
def create_tables(delete=False):
    connection = psycopg2.connect(**DB_CONFIG)
    cursor = connection.cursor()
    if delete:
        cursor.execute("DROP TABLE IF EXISTS ELN_WRITEUP_API_EXTRACT CASCADE")
        cursor.execute("DROP TABLE IF EXISTS ELN_WRITEUP_COMPARISON CASCADE")
    else:
        cursor.execute(
            """
            CREATE TABLE IF NOT EXISTS ELN_WRITEUP_API_EXTRACT (
                exp_id VARCHAR(7) NOT NULL,
                system_name VARCHAR(20) NOT NULL,
                write_up TEXT NOT NULL,
                summary_data TEXT NOT NULL,
                PRIMARY KEY(exp_id, system_name)
            );
        """
        )
        cursor.execute(
            """
            CREATE TABLE ELN_WRITEUP_COMPARISON (
                exp_id VARCHAR NOT NULL,
                system_name_1 VARCHAR NOT NULL,
                system_name_2 VARCHAR NOT NULL,
                diff TEXT,
                match_percentage NUMERIC,
                is_match BOOLEAN,
                PRIMARY KEY (exp_id, system_name_1, system_name_2),
                FOREIGN KEY (exp_id, system_name_1) REFERENCES eln_writeup_api_extract (exp_id, system_name),
                FOREIGN KEY (exp_id, system_name_2) REFERENCES eln_writeup_api_extract (exp_id, system_name)
            ); 
        """
        )
    connection.commit()
    cursor.close()
    connection.close()

In [55]:
async def fetch(url, headers):
    async with aiohttp.ClientSession() as session:
        async with session.get(url, headers=headers) as response:
            return await response.json()
            
async def save_writeup_to_db(exp_id, system_name, writeup, summary):
    """
    Saves writeup data to the database.

    Args:
        exp_id (str): Experiment ID.
        system_name (str): System name.
        writeup (str): The writeup content.
        summary (dict): Summary data.
    """
    async with DB_POOL.acquire() as conn:
        await conn.execute(
            """
            INSERT INTO ELN_WRITEUP_API_EXTRACT (exp_id, system_name, write_up, summary_data)
            VALUES ($1, $2, $3, $4)
            """,
            exp_id, system_name, writeup, summary
        )

async def save_compr_to_db(exp_id, system_name_1, system_name_2, diff, match_percentage, is_match):
    """
    Saves comparison data to the database.

    Args:
        exp_id (str): Experiment ID.
        system_name_1 (str): First system name.
        system_name_2 (str): Second system name.
        diff (str): Diff content.
        match_percentage (float): Match percentage.
        is_match (bool): Whether the match percentage meets the threshold.
    """
    async with DB_POOL.acquire() as conn:
        await conn.execute(
            """
            INSERT INTO ELN_WRITEUP_COMPARISON (exp_id, system_name_1, system_name_2, diff, match_percentage, is_match)
            VALUES ($1, $2, $3, $4, $5, $6)
            """,
            exp_id, system_name_1, system_name_2, diff, match_percentage, is_match
        )

In [36]:
async def process_exp_id(exp_id, token_dct):
    """
    Processes an individual experiment ID by fetching data, computing differences,
    and saving to the database.

    Args:
        exp_id (str): The experiment ID.
        token_dct (dict): Dictionary of tokens for authentication.
    """
    compr_data = {}
    for sname in SYS_NAMES:
        writeup_url_endpoint = f"https://{sname}.{BASE_URL}/studies/experiment/{exp_id}/writeup/{{includeHtml}}"
        headers = {"Authorization": f"Dotmatics {token_dct[sname]}"}
        writeup_data = await fetch(writeup_url_endpoint, headers)
        compr_data[sname] = {'writeup': writeup_data}

        dsid_string = "{0}_PROTOCOL,{0}_PROTOCOL_ID,{0}_ISID,{0}_CREATED_DATE".format(DS_IDS[sname]['summary'])
        exp_summary_endpoint = f"https://{sname}.{BASE_URL}/data/{DM_USER}/100000/{dsid_string}/{exp_id}"
        summary_data = await fetch(exp_summary_endpoint, headers)
        compr_data[sname]['summary'] = json.dumps(summary_data)

        await save_writeup_to_db(exp_id, sname, writeup_data, json.dumps(summary_data))

    writeup1 = compr_data[SYS_NAMES[0]]['writeup']
    writeup2 = compr_data[SYS_NAMES[1]]['writeup']
    diff = "\n".join(unified_diff(
        writeup1.splitlines(),
        writeup2.splitlines(),
        lineterm=''
    ))
    matcher = SequenceMatcher(None, writeup1, writeup2)
    match_percentage = matcher.ratio() * 100
    is_match = match_percentage >= 95

    await save_compr_to_db(exp_id, SYS_NAMES[0], SYS_NAMES[1], diff, match_percentage, is_match)

In [19]:
async def main():
    """
    Main function to handle the asynchronous logic for fetching, comparing,
    and saving data.
    """
    await init_db()
    tasks = []
    # get the prelude-masks exp ids
    DOMAIN = SYS_NAMES[1]
    async with aiohttp.ClientSession() as session:
        token_dct = {}
        for sname in SYS_NAMES:
            token_endpoint = f'https://{sname}.{BASE_URL}/authenticate/requestToken?isid={DM_USER}&password={DM_PASS}'
            token_dct[sname] = await fetch(token_endpoint, {})

        exp_id_query_endpoint = f"query/{DM_USER}/{PROJECT_ID}/{DS_IDS[DOMAIN]['exp_ids']}/EXPERIMENT_ID/greaterthan/1?limit={LIMIT}"
        url = f"https://{DOMAIN}.{BASE_URL}/{exp_id_query_endpoint}"
        headers = {"Authorization": f"Dotmatics {token_dct[DOMAIN]}"}
        exp_id_list = (await fetch(url, headers))["ids"]
        rev_exp_id_list = reversed(exp_id_list)

        for i, exp_id in enumerate(list(rev_exp_id_list)):
            tasks.append(process_exp_id(exp_id, token_dct))

        await asyncio.gather(*tasks)

    await DB_POOL.close()

In [39]:
create_tables(delete=True)

In [40]:
asyncio.run(main())

In [58]:
conn_str = f"postgresql+psycopg2://{DB_CONFIG['DB_USER']}:{DB_CONFIG['DB_PASS']}@{DB_CONFIG['DB_HOST']}:{DB_CONFIG['DB_PORT']}/{DB_CONFIG['DB_NAME']}"

def fetch_data_from_db(query, conn_str):
    engine = create_engine(conn_str)
    df = pd.read_sql(query, engine)
    return df
    
data_df = fetch_data_from_db("SELECT * FROM ELN_WRITEUP_API_EXTRACT", conn_str)
display(HTML(data_df.to_html()))

,exp_id,system_name,write_up,summary_data
0,189002,prelude,"To a solution ? (?, ? mmol){{9:row 1}}_XXXXX_ and ? (?, ? mmol){{9:row 2}}_XXXXX_ in MeCN (50 mL){{3:row 1}}_XXXXX_ was added ? (?, ? mmol){{9:row 3}}_XXXXX_ and stirred at 80 °C for 16 h under N2. HPLC(HPLC_MD01-032-P1A) showed 41.65% was remained, and a new peak formed. The reaction mixture was filtered and the solid washed with MeCN (10mL). The resulting solution was extracted with MTBE. The organic phase were combined and dried over Na2SO4, then filtered and the filtrate was concentrated under reduced pressure by water pump. The crude(3-(5-bromo-3-oxo-1H-isoindol-2-yl)piperidine-2,6-dione (2.6 g, 3.3552 mmol, 25.833% yield){{2:row 1}}_XXXXX_ ) was directly uesd to next step.","{""189002"": {""primary"": ""189002"", ""dataSources"": {""1426"": {""1"": {""CREATED_DATE"": ""29/10/2021"", ""ISID"": ""dmao"", ""PROTOCOL"": ""CRO_Affinity_Wuhan"", ""PROTOCOL_ID"": ""481""}}}}}"
1,188906,prelude,"To a 100 ml bottom flask was added #REF!, #REF!, #REF! and #REF!. The mixture was purged with nitrogen for 3 times and stirred for 2 min. The mixture was then added #REF!. The mixture was purged with nitrogen for 3 times. The resulting mixture was stirred under nitrogen at #REF! for 24 h. 4-Bromo-5-chloro-2-pyridin-2-amine remained and new peak was detected by HPLC. The mixture was stirred under nitrogen at #REF! for 24 h. 4-Bromo-5-chloro-2-pyridin-2-amine was consumed and new spots were detected by TLC (petroleum ether/ethyl acetate=2/1, Rf=0.55, 0.40). The mixture was filtered and the cake was washed with ethyl acetate (40 ml) and filtered. The filtrate was diluted with water (40 ml) and extracted with ethyl acetate (30 ml x 4). The combined organic layers were washed with brine (40 ml x 3), dried over sodium sulfate, filtered and the filtrate was concentrated under reduced pressure by water pump. The residue was purified by silica gel column chromatography (100-200 mesh, 0-20% ethyl acetate in petroleum ether) to give a product (0.6 g) as a yellow oil. But HNMR showed it was not the desired structure. The reaction was unsuccessful.","{""188906"": {""primary"": ""188906"", ""dataSources"": {""1426"": {""1"": {""CREATED_DATE"": ""28/10/2021"", ""ISID"": ""ppeng"", ""PROTOCOL"": ""CRO_Affinity_Wuhan"", ""PROTOCOL_ID"": ""481""}}}}}"
2,189090,prelude,"To a solution of #REF! in #REF! was added #REF! at #REF!. The resulting mixture was stirred at 25 °C for 40 h. 4-brompyridin-2-amin was consumed and new spots were detected by TLC (petroleum ether/ethyl acetate=3/1, Rf=0.30). The mixture was poured into sodium hydroxide aqueous solution (1 mol/L, 0 °C, 100 mL) and extracted with MTBE (120 ml x 3). The combined organic layers were washed with water (120 ml x 2), brine (120 ml x 2), dried over sodium sulfate, filtered and the filtrate was concentrated under reduced pressure by water pump. The residue was suspended in dichloromethane (45 ml) and stirred for 30 min. The mixture was filtered and the cake was washed with petroleum ether (30 ml) and filtered. The cake was dried under reduced pressure by water pump to give #REF! as a grey solid. The filtrate was concentrated under reduced pressure by water pump. The residue was purified by silica gel colulmn chromatography (100-200 mesh, 0-14% ethyl acetate in petroleum ether) to give #REF! as a grey solid.","{""189090"": {""primary"": ""189090"", ""dataSources"": {""1426"": {""1"": {""CREATED_DATE"": ""01/11/2021"", ""ISID"": ""ppeng"", ""PROTOCOL"": ""CRO_Affinity_Wuhan"", ""PROTOCOL_ID"": ""481""}}}}}"
3,189091,prelude,"To a solution of #REF! in #REF! was added #REF! dropwise. The resulting mixture was stirred at #REF! for 20 h. N-(3-amino-4-methylthiophen-2-yl)-N-propan-2-ylacetamide was consumed and new peak was detected by HPLC. The mixture was poured into 10% sodium carbonate aqueous solution (0 °C , 15 ml) and stirred for 10 min. The mixture was extracted with ethyl acetate (10 ml x 3). The combined organic layers were washed with 

In [59]:
with open("comparisons.html", "w") as f:
    f.write(data_df.to_html())